This notebook is dedicated to the extraction and pre-processing of relevant data for the cartography and dynamic analysis of territorial polarisation in 1980-2020 France. 

In [1]:
### Importing libraries ###

import numpy as np
import json
import pandas as pd
import geopandas as gpd
import datetime

In [2]:
### Importing communal geo data ###

communal_geo = gpd.read_file("Raw data/communes-version-simplifiee.geojson")

In [3]:
### Building & exporting ZE1990 geography ###

with open("Raw data/ZE1990_comp.csv") as ZE1990_comp_csv :
    ZE1990_comp = pd.read_csv(ZE1990_comp_csv, header=1, usecols=[0,4], names=["Code","ZE1990"])

ZE1990_geo = communal_geo.merge(ZE1990_comp,right_on="Code",left_on="code")
ZE1990_geo = ZE1990_geo[["ZE1990","geometry"]]
ZE1990_geo = ZE1990_geo[ZE1990_geo.geometry.notnull()]
ZE1990_geo = ZE1990_geo.dissolve("ZE1990")
ZE1990_geo = ZE1990_geo.reset_index()
ZE1990_geo = ZE1990_geo.to_crs({'init': 'epsg:3857'})

    ## Export
ZE1990_geo.to_file("Output/ZE1990_geo.geojson", driver="GeoJSON", index = "ZE1990")

    ## Setting index
ZE1990_geo = ZE1990_geo.set_index("ZE1990")

In [4]:
### Building & exporting ZE2010 geography ###
    
with open("Raw data/ZE2010_comp.csv") as ZE2010_comp_csv :
    ZE2010_comp = pd.read_csv(ZE2010_comp_csv, header=5, usecols=[0,2], names=["Code","ZE2010"])

ZE2010_geo = communal_geo.merge(ZE2010_comp,right_on="Code",left_on="code")
ZE2010_geo = ZE2010_geo[["ZE2010","geometry"]]
ZE2010_geo = ZE2010_geo[ZE2010_geo.geometry.notnull()]
ZE2010_geo = ZE2010_geo.dissolve("ZE2010")
ZE2010_geo = ZE2010_geo.reset_index()
ZE2010_geo = ZE2010_geo.to_crs({'init': 'epsg:3857'})

    ## Export
ZE2010_geo.to_file("Output/ZE2010_geo.geojson", driver="GeoJSON")

    ## Setting index
ZE2010_geo = ZE2010_geo.set_index("ZE2010")

In [5]:
print(ZE1990_geo.crs)

{'init': 'epsg:3857'}


Coordinate system is a Mercator projection. As the information we need is summarised in the relative distance/areas of ZE, and France is latitudinally small, I am going to approximate this measures to meters.

In [6]:
### Building & exporting geographical data for ZE1990

area_ZE1990 = ZE1990_geo.geometry.area/10**6
area_ZE1990.to_json("Output/area_ZE1990.json")

dist_matrix_ZE1990 = pd.DataFrame(columns=ZE1990_geo.index, index=ZE1990_geo.index)
dist_matrix_ZE1990 = dist_matrix_ZE1990.apply(lambda x: ZE1990_geo.geometry.centroid.distance(ZE1990_geo.loc[x.name].geometry.centroid)/10**3)
dist_matrix_ZE1990.to_json("Output/dist_matrix_ZE1990.json")


In [7]:
### Building & exporting distance matrix for ZE2010

area_ZE2010 = ZE2010_geo.geometry.area/10**6
area_ZE2010.to_json("Output/area_ZE2010.json")

dist_matrix_ZE2010 = pd.DataFrame(columns=ZE2010_geo.index, index=ZE2010_geo.index)
dist_matrix_ZE2010 = dist_matrix_ZE2010.apply(lambda x: ZE2010_geo.geometry.centroid.distance(ZE2010_geo.loc[x.name].geometry.centroid)/10**3)
dist_matrix_ZE2010.to_json("Output/dist_matrix_ZE2010.json")


In [8]:
print(area_ZE1990)

ZE1990
1131     244.907819
1132     890.062606
1133     322.590612
1134     102.842606
1135    1923.896535
           ...     
9412    2481.469872
9414    2563.220393
9415    2005.960848
9416    2267.591032
9417     789.922325
Length: 348, dtype: float64


In [9]:
### Building & exporting FILO timeseries from 2001 to 2008
 
dates = range(2001,2009)

dUC_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
med_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
d1_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
d9_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
intQ_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
moy_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)

for i in dates:
    with open("Raw data/FILOSOFI - Revenus déclarés par ZE1990 - " + str(i) + ".csv") as ZE_revenue_csv :

        ## load from 2001 to 2006        
        if i in range(2001,2007):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,5,7,10,17,18,20], names = ["ZE1990","nUC","med","d1","d9","intQ","moy"])

        ## load from 2007 to 2008
        elif i in range(2007,2009):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,3,5,8,15,16,18], names = ["ZE1990","nUC","med","d1","d9","intQ","moy"])
            
        ZE_data = ZE_data.set_index("ZE1990")        
        
        dUC_timeseries_ZE1990.loc[i]=ZE_data["nUC"]/area_ZE1990
        med_timeseries_ZE1990.loc[i]=ZE_data["med"]
        d1_timeseries_ZE1990.loc[i]=ZE_data["d1"]
        d9_timeseries_ZE1990.loc[i]=ZE_data["d9"]
        intQ_timeseries_ZE1990.loc[i]=ZE_data["intQ"]
        moy_timeseries_ZE1990.loc[i]=ZE_data["moy"]
                
dUC_timeseries_ZE1990.to_json("Output/dUC_timeseries_ZE1990.json")
med_timeseries_ZE1990.to_json("Output/med_timeseries_ZE1990.json")
d1_timeseries_ZE1990.to_json("Output/d1_timeseries_ZE1990.json")
d9_timeseries_ZE1990.to_json("Output/d9_timeseries_ZE1990.json")
intQ_timeseries_ZE1990.to_json("Output/intQ_timeseries_ZE1990.json")
moy_timeseries_ZE1990.to_json("Output/moy_timeseries_ZE1990.json")


In [10]:
### Building & exporting FILO timeseries from 2009 to 2016
 
dates = range(2009,2018)

dUC_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
med_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
d1_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
d9_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
intQ_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
## moy_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)


for i in dates:
    with open("Raw data/FILOSOFI - Revenus déclarés par ZE2010 - " + str(i) + ".csv") as ZE_revenue_csv :
        
        ## load from 2009 to 2011        
        if i in range(2009,2012):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,3,5,8,15,16,18], names = ["ZE2010","nUC","med","d1","d9","intQ","moy"])
        
        ## load from 2012 to 2016
        elif i in range(2012,2018):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 5, usecols=[0,4,7,10,17,18], names = ["ZE2010","nUC","med","d1","d9","intQ"])

        ZE_data = ZE_data.set_index("ZE2010")

        dUC_timeseries_ZE2010.loc[i]=ZE_data["nUC"]/area_ZE2010
        med_timeseries_ZE2010.loc[i]=ZE_data["med"]
        d1_timeseries_ZE2010.loc[i]=ZE_data["d1"]
        d9_timeseries_ZE2010.loc[i]=ZE_data["d9"]
        intQ_timeseries_ZE2010.loc[i]=ZE_data["intQ"]
##        moy_timeseries_ZE2010.loc[i]=ZE_data["moy"]

        
        
dUC_timeseries_ZE2010.to_json("Output/dUC_timeseries_ZE2010.json")
med_timeseries_ZE2010.to_json("Output/med_timeseries_ZE2010.json")
d1_timeseries_ZE2010.to_json("Output/d1_timeseries_ZE2010.json")
d9_timeseries_ZE2010.to_json("Output/d9_timeseries_ZE2010.json")
intQ_timeseries_ZE2010.to_json("Output/intQ_timeseries_ZE2010.json")
##moy_timeseries_ZE2010.to_json("Output/moy_timeseries_ZE2010.json")

In [11]:
### Building sectorial composition timeseries

with open("Raw data/ESTEL - emploi sectoriel par ZE - 1998 à 2017.csv") as sectcomp_csv:
    sectcomp_data = pd.read_csv(sectcomp_csv, header=5)
    sectcomp_data = sectcomp_data.drop(sectcomp_data[sectcomp_data["Activite"] == "TT-Total"].index)

with open("Raw data/ESTEL - emploi non salarié par ZE - 1998 à 2017.csv") as nonsalcomp_csv:
    nonsalcomp_data = pd.read_csv(nonsalcomp_csv, header=5)
    nonsalcomp_data["Activite"] = "NS-Non salarie"
    
sectcomp_timeseries_ZE2010 = pd.concat([sectcomp_data, nonsalcomp_data], sort=True)

    ## Setting index structure and format
sectcomp_timeseries_ZE2010["ZE2010"] = sectcomp_timeseries_ZE2010["Zone_emploi"].apply(lambda x: int(str(x)[0:4]))
sectcomp_timeseries_ZE2010 = sectcomp_timeseries_ZE2010.drop(["Region", "Zone_emploi"], axis=1)
sectcomp_timeseries_ZE2010 = sectcomp_timeseries_ZE2010.set_index(["ZE2010", "Activite"]).sort_index()
sectcomp_timeseries_ZE2010 = sectcomp_timeseries_ZE2010.T
sectcomp_timeseries_ZE2010.index = sectcomp_timeseries_ZE2010.index.astype(int)

    ## Formatting and normalising
sectcomp_timeseries_ZE2010 = sectcomp_timeseries_ZE2010.applymap(lambda x: int(str(x).replace(",", "")))
sectcomp_timeseries_ZE2010 = sectcomp_timeseries_ZE2010/sectcomp_timeseries_ZE2010.groupby(level=0, axis=1).sum()


In [12]:
### Building and exporting tertiarisation index timeseries

tert_timeseries_ZE2010 = sectcomp_timeseries_ZE2010.xs("GU-Tertiaire marchand", level="Activite", axis=1) + sectcomp_timeseries_ZE2010.xs("OQ-Tertiaire non marchand", level="Activite", axis=1)
tert_timeseries_ZE2010.to_json("Output/tert_timeseries_ZE2010.json")

In [13]:
### Building & exporting fonctional timeseries from 2001 to 2008

dates = [1999, 2006]

emp_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
fonc_timeseries_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)

for i in dates:
    with open("Raw data/Analyse fonctionnelle - Données communales - " + str(i) + ".csv") as communal_fonc_csv :
        communal_fonc = pd.read_csv(communal_fonc_csv, header = 10, usecols = [0,4,25], names = ["Code", "Employment","CFM"], dtype = str)        

        communal_fonc = communal_fonc.merge(ZE1990_comp,on="Code")
        communal_fonc = communal_fonc.drop("Code", axis=1)
        
        communal_fonc[["Employment","CFM"]] = communal_fonc[["Employment","CFM"]].apply(pd.to_numeric, errors="coerce")        
        communal_fonc = communal_fonc.groupby(["ZE1990"]).sum()
        
        emp_timeseries_ZE1990.loc[i] = communal_fonc["Employment"]
        fonc_timeseries_ZE1990.loc[i] = communal_fonc["CFM"]
        
emp_timeseries_ZE1990.to_json("Output/emp_timeseries_ZE1990.json")
fonc_timeseries_ZE1990.to_json("Output/fonc_timeseries_ZE1990.json")

In [14]:
### Building & exporting fonctionnal timeseries from 2009 to 2016

dates = [2011, 2016]

emp_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
fonc_timeseries_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)

for i in dates:
    with open("Raw data/Analyse fonctionnelle - Données communales - " + str(i) + ".csv") as communal_fonc_csv :
        communal_fonc = pd.read_csv(communal_fonc_csv, header = 10, usecols = [0,4,25], names = ["Code", "Employment","CFM"], dtype = str)        

        communal_fonc = communal_fonc.merge(ZE2010_comp,on="Code")
        communal_fonc = communal_fonc.drop("Code", axis=1)

        communal_fonc[["Employment","CFM"]] = communal_fonc[["Employment","CFM"]].apply(pd.to_numeric, errors="coerce")        
        communal_fonc = communal_fonc.groupby(["ZE2010"]).sum()
        
        emp_timeseries_ZE2010.loc[i] = communal_fonc["Employment"]
        fonc_timeseries_ZE2010.loc[i] = communal_fonc["CFM"]
        
emp_timeseries_ZE2010.to_json("Output/emp_timeseries_ZE2010.json")
fonc_timeseries_ZE2010.to_json("Output/fonc_timeseries_ZE2010.json")

In [15]:
### Defining timeseries to growthseries function

def to_growth(timeseries):
    
    growthseries = timeseries
    growthseries = growthseries.applymap(lambda x: np.log(x))
    growthseries = growthseries.apply(lambda x: x.shift(-1) - x)
    
    growthseries = growthseries.dropna(axis=0,how="all").dropna(axis=1,how="any")
    
    return(growthseries)

In [16]:
### Building & exporting growthseries

to_growth(dUC_timeseries_ZE1990).to_json("Output/dUC_growthseries_ZE1990.json")
to_growth(med_timeseries_ZE1990).to_json("Output/med_growthseries_ZE1990.json")
to_growth(intQ_timeseries_ZE1990).to_json("Output/intQ_growthseries_ZE1990.json")
to_growth(emp_timeseries_ZE1990).to_json("Output/emp_growthseries_ZE1990.json")
to_growth(fonc_timeseries_ZE1990).to_json("Output/fonc_growthseries_ZE1990.json")

to_growth(dUC_timeseries_ZE2010).to_json("Output/dUC_growthseries_ZE2010.json")
to_growth(med_timeseries_ZE2010).to_json("Output/med_growthseries_ZE2010.json")
to_growth(intQ_timeseries_ZE2010).to_json("Output/intQ_growthseries_ZE2010.json")
to_growth(emp_timeseries_ZE2010).to_json("Output/emp_growthseries_ZE2010.json")
to_growth(fonc_timeseries_ZE2010).to_json("Output/fonc_growthseries_ZE2010.json")


/home/avel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [17]:
### Building and exporting financiarisation proxies

with open("Raw data/Banque de France - % étranger de la dette publique.csv") as foreigndebt_csv :
    foreigndebt_timeseries = pd.read_csv(foreigndebt_csv, usecols=[0,1], header = 5, index_col = 0)
    foreigndebt_timeseries.index = pd.MultiIndex.from_arrays([foreigndebt_timeseries.index.str[-4:], foreigndebt_timeseries.index.str[:2]])
    foreigndebt_timeseries = foreigndebt_timeseries.groupby(level=0).mean()
    
foreigndebt_timeseries.to_json("Output/foreigndebt_timeseries.json")


with open("Raw data/Banque de France - % étranger des capitaux du CAC 40.csv") as foreigncapitals_csv :
    foreigncapitals_timeseries = pd.read_csv(foreigncapitals_csv, usecols = [0,1], header = 5, index_col=0)

foreigncapitals_timeseries.to_json("Output/foreigncapitals_timeseries.json")